In [1]:
import pandas as pd
from sqlalchemy import create_engine, inspect
from config import user, password, db_name

# Extract

In [2]:
# Import CSV with AirBnB data
csv_file = "Resources/listings.csv"
airbnb_data_df = pd.read_csv(csv_file)
airbnb_data_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2384,"Hyde Park - Walk to UChicago, 10 min to McCormick",2613,Rebecca,NaN,Hyde Park,41.78790,-87.58780,Private room,60,2,178,2019-12-15,2.56,1,353
1,4505,394 Great Reviews. 127 y/o House. 40 yds to tr...,5775,Craig & Kathleen,NaN,South Lawndale,41.85495,-87.69696,Entire home/apt,105,2,395,2020-07-14,2.81,1,155
2,7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,NaN,West Town,41.90289,-87.68182,Entire home/apt,60,2,384,2020-03-08,2.81,1,321
3,9811,Barbara's Hideaway - Old Town,33004,At Home Inn,NaN,Lincoln Park,41.91769,-87.63788,Entire home/apt,65,4,49,2019-10-23,0.63,9,300
4,10610,3 Comforts of Cooperative Living,2140,Lois,NaN,Hyde Park,41.79612,-87.59261,Private room,21,1,44,2020-02-14,0.61,5,168


In [3]:
# Import CSV with community area numbers
csv_file = "Resources/chicago-community-areas.csv"
communities_df = pd.read_csv(csv_file)
communities_df.head()

,Community Area,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,75,76,77
0,name,Rogers Park,West Ridge,Uptown,Lincoln Square,North Center,Lake View,Lincoln Park,Near North Side,Edison Park,...,Englewood,Greater Grand Crossing,Ashburn,Auburn Gresham,Beverly,Washington Heights,Mount Greenwood,Morgan Park,O'Hare,Edgewater


In [30]:
# Import CSV with Chicago 2013 population numbers
csv_file = "Resources/chicago_population2013.csv"
population_df = pd.read_csv(csv_file, sep=';')
population_df.head()

,Community Area,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,75,76,77
0,name,Rogers Park,West Ridge,Uptown,Lincoln Square,North Center,Lake View,Lincoln Park,Near North Side,Edison Park,...,Englewood,Greater Grand Crossing,Ashburn,Auburn Gresham,Beverly,Washington Heights,Mount Greenwood,Morgan Park,O'Hare,Edgewater
1,population,54991,71942,56362,39493,31867,94368,64116,80484,11187,...,30654,32602,41081,48743,20034,26493,19093,22544,12756,56521
2,income,39482,47323,40324,57749,81524,70746,82707,76290,77678,...,19743,29663,62238,34767,83092,42053,80505,56886,49601,43331
3,latinos,0.244,0.204,0.142,0.191,0.136,0.076,0.056,0.049,0.078,...,0.011,0.012,0.368,0.009,0.046,0.01,0.072,0.027,0.095,0.165
4,blacks,0.263,0.111,0.2,0.038,0.023,0.039,0.043,0.108,0.003,...,0.974,0.969,0.462,0.978,0.341,0.974,0.052,0.667,0.032,0.143


# Transform

### AirBnB DataFrame

In [5]:
airbnb_data_df = airbnb_data_df.drop(["neighbourhood_group"], axis=1)

In [6]:
airbnb_data_df.set_index("id", inplace=True)

In [7]:
airbnb_data_df.dropna(how = "any", inplace=True)

### Listing and Location DataFrames - Split from AirBnB DataFrame

In [8]:
listing_info_df = airbnb_data_df.iloc[:, [1, 2, 6, 7, 8, 9, 10, 11, 12, 13]]

location_info_df = airbnb_data_df.iloc[:, [0, 3, 4, 5]]

In [9]:
location_info_df = location_info_df.rename(columns={"name": "address", "neighbourhood": "neighborhood"})

In [10]:
listing_info_df.index = listing_info_df.index.astype(int)

In [11]:
location_info_df["neighborhood"] = location_info_df["neighborhood"].str.lower()
location_info_df["neighborhood"] = location_info_df["neighborhood"].str.replace("\'", "")

In [49]:
location_info_df
listing_info_df

,host_id,host_name,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
id,,,,,,,,,,
2384,2613,Rebecca,Private room,60,2,178,2019-12-15,2.56,1,353
4505,5775,Craig & Kathleen,Entire home/apt,105,2,395,2020-07-14,2.81,1,155
7126,17928,Sarah,Entire home/apt,60,2,384,2020-03-08,2.81,1,321
9811,33004,At Home Inn,Entire home/apt,65,4,49,2019-10-23,0.63,9,300
10610,2140,Lois,Private room,21,1,44,2020-02-14,0.61,5,168
...,...,...,...,...,...,...,...,...,...,...
45351578,77382816,Edmund,Entire home/apt,67,1,2,2020-09-20,2.00,1,307
45368527,128265803,Victor,Private room,24,1,2,2020-09-20,2.00,4,14
45386114,324740940,Andrea,Entire home/apt,97,1,2,2020-09-19,2.00,4,81


### Communities DataFrame

In [13]:
communities_df.columns = communities_df.columns.str.lower()

# Replace spaces in column names with underscores
communities_df.columns = communities_df.columns.str.replace(' ', '_')

communities_df.set_index("community_area", inplace=True)

In [14]:
communities_df = communities_df.transpose()

In [15]:
communities_df = communities_df.rename(columns={"name": "neighborhood"})

In [16]:
communities_df.index = communities_df.index.astype(int)

In [17]:
communities_df["neighborhood"] = communities_df["neighborhood"].str.lower()
communities_df["neighborhood"] = communities_df["neighborhood"].str.replace("\'", "")
communities_df

community_area,neighborhood
1,rogers park
2,west ridge
3,uptown
4,lincoln square
5,north center
...,...
73,washington heights
74,mount greenwood
75,morgan park
76,ohare


### Population DataFrame

In [31]:
# Make the column names lowercase
population_df.columns = population_df.columns.str.lower()

# Replace spaces in column names with underscores
population_df.columns = population_df.columns.str.replace(' ', '_')

In [32]:
population_df.set_index("community_area", inplace=True)
population_df = population_df.transpose()

In [33]:
population_df = population_df.rename(columns={"name": "neighborhood"})

In [37]:
population_df["neighborhood"] = population_df["neighborhood"].str.lower()

# Replace spaces in column names with underscores
population_df["neighborhood"] = population_df["neighborhood"].str.replace("\'", "")

In [43]:
population_df.index = population_df.index.astype(int)

In [44]:
population_df

community_area,neighborhood,population,income,latinos,blacks,white,asian,other
1,rogers park,54991,39482,0.244,0.263,0.393,0.064,0.036
2,west ridge,71942,47323,0.204,0.111,0.427,0.225,0.032
3,uptown,56362,40324,0.142,0.2,0.516,0.114,0.028
4,lincoln square,39493,57749,0.191,0.038,0.631,0.111,0.029
5,north center,31867,81524,0.136,0.023,0.773,0.045,0.022
...,...,...,...,...,...,...,...,...
73,washington heights,26493,42053,0.01,0.974,0.005,0,0.012
74,mount greenwood,19093,80505,0.072,0.052,0.86,0.007,0.01
75,morgan park,22544,56886,0.027,0.667,0.287,0.004,0.014
76,ohare,12756,49601,0.095,0.032,0.772,0.083,0.019


### Load

In [24]:
# Create database connection
rds_connection_string = f"{user}:{password}@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [50]:
# Check for tables
inspector = inspect(engine)
print(inspector.get_table_names())

['listing', 'airbnb_loc', 'communities', 'population']


In [66]:
# Load CSV converted DataFrame into database
communities_df.to_sql(name='communities', con=engine, if_exists='append', index=False)

In [67]:
# Load CSV converted DataFrame into database
listing_info_df.to_sql(name='listing', con=engine, if_exists='append', index=True)
#resultDf.to_sql('table_name', engine, schema="schema_name", if_exists="append", index=False)


In [68]:
# Load CSV converted DataFrame into database
location_info_df.to_sql(name='airbnb_loc', con=engine, if_exists='append', index=True)

In [46]:
population_df.to_sql(name='population', con=engine, if_exists='append', index=False)

In [47]:
# Confirm data has been added
pd.read_sql_query('select * from listing', con=engine).head()

,id,host_id,host_name,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2384,2613,Rebecca,Private room,$60.00,2,178,2019-12-15,2.56,1,353
1,4505,5775,Craig & Kathleen,Entire home/apt,$105.00,2,395,2020-07-14,2.81,1,155
2,7126,17928,Sarah,Entire home/apt,$60.00,2,384,2020-03-08,2.81,1,321
3,9811,33004,At Home Inn,Entire home/apt,$65.00,4,49,2019-10-23,0.63,9,300
4,10610,2140,Lois,Private room,$21.00,1,44,2020-02-14,0.61,5,168


In [ ]:
# Load CSV converted DataFrame into database
df.to_sql(name='table_name', con=engine, if_exists='append', index=False)